In [1]:
###We will update and install the necessary Python libraries.

##%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 692.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 29.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 168.6 kB/s eta 0:00:00 0:00:01m
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
Note:

In [2]:
####save Hugging Face and Weights and Biases API keys and access them in the Kaggle notebook.

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HUGGING_FACE_WRITE")
secret_value_1 = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_value_2 = user_secrets.get_secret("wandb")

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

2024-08-12 17:58:42.839004: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 17:58:42.839117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 17:58:42.941150: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
###  We will use the Hugging Face API to save and push the model to the Hugging Face Hub.
!huggingface-cli login --token $secret_value_0

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
wandb.login(key = secret_value_2)
run = wandb.init(
    project='Fine tuning mistral 7B', 
    job_type="training", 
    anonymous="allow"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: muhammadnadeem (muhammadnadeem-gift-university). Use `wandb login --relogin` to force relogin


In [6]:
base_model = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"
dataset_name = "/kaggle/input/mlec-english-sample/train.csv"
new_model = "mistral_7b_mlec"

In [7]:
## function to conver binary label into text labels
def convert_binary_to_label(data,emotion_labels):
  
    """Converts binary emotion values to corresponding labels.
      Args:
        data: A list of lists, where the elements are binary values representing presence of emotion in a tweet.

      Returns:
        A list of lists, where each list contains the tweet and a list of emotion labels.
    """

    result = []
    for row in data:
            tweet = row[0]
            emotions = [label for label, value in zip(emotion_labels, row[1:]) if int(value) == 1]
            if not emotions:
                  emotions.append('neutral')  # Add 'neutral' if the list is empty
            text_string = ', '.join(emotions)
            result.append((text_string))

    return result

In [8]:
##### function to conver data into mistral train data format
def convert_mistral_promt(my_df):
    promtlist = []
    for index, row in my_df.iterrows():
        promt= f"""\' <s>[INST] Being an expert in analyzing scientific research,
        you are required to categorize the research paper titled "{row['TITLE']}".
        The abstract is as follows:
        "{row['ABSTRACT']}"
        
        Based on the abstract, determine the relevant categories from the following disciplines:
        - computer_science
        - physics
        - mathematics
        - statistics
        - quantitative_biology
        - quantitative_finance
        
        Your task is to identify and list the applicable disciplines for the given research paper.
        If the paper doesn't fit into any of the provided disciplines, return ['None'].
        
        Your answer should be in the form of a Python list containing the relevant disciplines.
        
        Example: ['computer_science', 'mathematics']

        [/INS] "{[row['computer_science'], row['physics'], row['mathematics'], row['statistics'], row['quantitative_biology'], row['quantitative_finance']]}" </s>\' """
        promtlist.append(promt)

    # Return the list of prompts
    return promtlist


In [9]:
import csv
import pandas as pd
import numpy as np
import random
from datasets import Dataset, DatasetDict
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split

In [10]:
!pip install iterative-stratification

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [11]:
import random
import csv
import numpy as np
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from datasets import Dataset

# Predefined list for disciplines corresponding to the label columns
test_labels = ['computer_science', 'physics', 'mathematics', 'statistics', 'quantitative_biology', 'quantitative_finance']

# Load the data
with open('/kaggle/input/mlec-english-sample/train.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile, delimiter=','))
    header_row = data.pop(0)

# Shuffle data
random.seed(0)
random.shuffle(data)

# Extract the text and labels
text = [row[2].strip() for row in data]  # Extract the ABSTRACT from the third column (index 2)
labels = []
for row in data:
    try:
        # Convert label values to integers, assuming they are binary (0 or 1)
        # We skip the first two columns (ID and TITLE) and only process label columns
        label_row = list(map(int, row[3:]))  # Extract labels starting from the fourth column (index 3)
        labels.append(label_row)
    except ValueError:
        print(f"Error converting row: {row}")  # Add error handling to identify problematic rows
        continue

labels = np.array(labels, dtype=int)

# Stratified train-test split for multilabel dataset
mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=0)

for train_index, val_index in mskf.split(np.zeros(len(labels)), labels):
    x_train, x_val = [text[i] for i in train_index], [text[i] for i in val_index]
    y_train, y_val = labels[train_index], labels[val_index]
    break  # Since we only need one split

# Convert binary labels to actual labels
def convert_binary_to_label(binary_labels, label_list):
    labels = []
    for label_set in binary_labels:
        label_names = [label_list[i] for i, value in enumerate(label_set) if value == 1]
        labels.append(label_names if label_names else ['None'])
    return labels

# Convert binary labels to actual labels
train_act_lab = convert_binary_to_label(y_train, test_labels)
val_act_lab = convert_binary_to_label(y_val, test_labels)

# Create DataFrames for the training and validation data
train_df = pd.DataFrame({'TITLE': x_train, 'ABSTRACT': train_act_lab})
val_df = pd.DataFrame({'TITLE': x_val, 'ABSTRACT': val_act_lab})

# Convert DataFrames into prompts
train_prompt = convert_mistral_promt(train_df)
val_prompt = convert_mistral_promt(val_df)

# Convert lists to datasets
train_dataset = Dataset.from_dict({'text': train_prompt})
val_dataset = Dataset.from_dict({'text': val_prompt})


KeyError: 'computer_science'

In [12]:
import csv
import pandas as pd
import numpy as np
import random
from datasets import Dataset
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# Function to convert data into Mistral train data format
def convert_mistral_prompt(my_df):
    prompt_list = []
    for index, row in my_df.iterrows():
        prompt = f"""\'<s>[INST] Being an expert in analyzing scientific research,
        you are required to categorize the research paper titled "{row['TITLE']}".
        The abstract is as follows:
        "{row['ABSTRACT']}"
        
        Based on the abstract, determine the relevant categories from the following disciplines:
        - computer_science
        - physics
        - mathematics
        - statistics
        - quantitative_biology
        - quantitative_finance
        
        Your task is to identify and list the applicable disciplines for the given research paper.
        If the paper doesn't fit into any of the provided disciplines, return ['None'].
        
        Your answer should be in the form of a Python list containing the relevant disciplines.
        
        Example: ['computer_science', 'mathematics']

        [/INS] {row['Labels']} </s>\'"""
        prompt_list.append(prompt)

    # Return the list of prompts
    return prompt_list

# Predefined list for disciplines corresponding to the label columns
test_labels = ['computer_science', 'physics', 'mathematics', 'statistics', 'quantitative_biology', 'quantitative_finance']

# Load the data
with open('/kaggle/input/mlec-english-sample/train.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile, delimiter=','))
    header_row = data.pop(0)

# Shuffle data
random.seed(0)
random.shuffle(data)

# Extract the text and labels
text = [row[2].strip() for row in data]  # Extract the ABSTRACT from the third column (index 2)
labels = []
for row in data:
    try:
        # Convert label values to integers, assuming they are binary (0 or 1)
        # We skip the first two columns (ID and TITLE) and only process label columns
        label_row = list(map(int, row[3:]))  # Extract labels starting from the fourth column (index 3)
        labels.append(label_row)
    except ValueError:
        print(f"Error converting row: {row}")  # Add error handling to identify problematic rows
        continue

labels = np.array(labels, dtype=int)

# Stratified train-test split for multilabel dataset
mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=0)

for train_index, val_index in mskf.split(np.zeros(len(labels)), labels):
    x_train, x_val = [text[i] for i in train_index], [text[i] for i in val_index]
    y_train, y_val = labels[train_index], labels[val_index]
    break  # Since we only need one split

# Convert binary labels to actual labels
def convert_binary_to_label(binary_labels, label_list):
    labels = []
    for label_set in binary_labels:
        label_names = [label_list[i] for i, value in enumerate(label_set) if value == 1]
        labels.append(label_names if label_names else ['None'])
    return labels

# Convert binary labels to actual labels
train_act_lab = convert_binary_to_label(y_train, test_labels)
val_act_lab = convert_binary_to_label(y_val, test_labels)

# Create DataFrames for the training and validation data
train_df = pd.DataFrame({'TITLE': x_train, 'ABSTRACT': [text[i] for i in train_index], 'Labels': train_act_lab})
val_df = pd.DataFrame({'TITLE': x_val, 'ABSTRACT': [text[i] for i in val_index], 'Labels': val_act_lab})

# Convert DataFrames into prompts
train_prompt = convert_mistral_prompt(train_df)
val_prompt = convert_mistral_prompt(val_df)

# Convert lists to datasets
train_dataset = Dataset.from_dict({'text': train_prompt})
val_dataset = Dataset.from_dict({'text': val_prompt})


In [14]:
#################################
###Loading the Mistral 7B model
#################################

bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        #load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
##Loading the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

In [16]:
####Adding the adopter to the layer
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [17]:
#####Hyperparmeters

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    logging_dir="./logs",
    metric_for_best_model="eval_loss",
    learning_rate=1e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
##############SFT parameters

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn

Map:   0%|          | 0/18861 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2111 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [20]:
#new_model = "mistral_7b_mlec_eng_1"
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

wandb.finish()
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_

In [21]:
try:
    trainer.model.push_to_hub(new_model, use_temp_dir=False)
    tokenizer.push_to_hub(new_model, use_temp_dir=False)
except:
    print("An exception occurred")

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

In [22]:
# HuggingFace pipeline
#llm = HuggingFacePipeline(pipeline=pipe)

import re
# Define the emotion labels
emotion_labels = ['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']



In [23]:
def extract_emotion(text):
    """Extracts text outside [INST] and [/INST] tags."""
    last_inst_index = text.rfind("[/INST]")
    if last_inst_index != -1:
        my_text = text[last_inst_index + len("[/INST]") :]
        my_text= my_text.lower()
        #print(my_text, "%%%%%%%%")
        response_text = my_text.replace('[', ' ').replace('\n',' ').replace(']', ' ').replace('\'', ' ').replace('.',' ').replace('\"', ' ').replace('#',' ')
        detected_emotions = response_text.split(" ")
        #print('***********',detected_emotions)
        return [1 if emotion in detected_emotions else 0 for emotion in emotion_labels]
    else:
        return [0] * len(emotion_labels)



In [24]:
#/kaggle/input/mlec-english-sample/Train_Data_Sample.csv

import pandas as pd
import csv
data= pd.read_csv('/kaggle/input/mlec-english-sample/Test_Data_Sample.csv')
data_t = data[14:16].copy()
#Extract tweet text from dataset
test_texts = data_t['Tweet'].tolist()

#Extract actual_values for emotions
test_labels = data_t.drop(columns=['Tweet']).values

In [25]:
logging.set_verbosity(logging.CRITICAL)

predictions = []
for text in test_texts:
    query = f"""[INST]Being an expert who can detect emotion
        present in a given text enclosed in “” please read the provided Urdu text. 
        Identify and list up to four prominent emotions
        present in the provided Urdu text. 
        Consider the following emotion labels: 
        anger
        anticipation
        disgust
        fear
        joy
        love
        optimism
        pessimism
        sadness
        surprise
        trust
        neutral. 
        If no emotions are detected or the text is unclear, return ['neutral']. 
        your answer should be in the form of a Python List containing emotion labels only mentioned above.

        The text is : \"{text}\" [/INST] Emotion: """
    
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=20 )
    result = pipe(query)
    print(result[0]['generated_text'])
    #result = llm(query)
    #display(Markdown(f"<b>{query.removeprefix('[INST]').removesuffix('[/INST]')}</b>"))
    #display(Markdown(f"<p>{result[0]['generated_text']}</p>"))
    r_label = extract_emotion(result[0]['generated_text'])
    print("****", r_label)
    predictions.append(r_label)

[INST]Being an expert who can detect emotion
        present in a given text enclosed in “” please read the provided Urdu text. 
        Identify and list up to four prominent emotions
        present in the provided Urdu text. 
        Consider the following emotion labels: 
        anger
        anticipation
        disgust
        fear
        joy
        love
        optimism
        pessimism
        sadness
        surprise
        trust
        neutral. 
        If no emotions are detected or the text is unclear, return ['neutral']. 
        your answer should be in the form of a Python List containing emotion labels only mentioned above.

        The text is : "The bitter the battle, the sweeter the victory..." [/INST] Emotion:  'anger, love'  ii 'anger, disgust, love'  iii 'anger,
**** [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[INST]Being an expert who can detect emotion
        present in a given text enclosed in “” please read the provided Urdu text. 
        Identify and list up to

In [26]:
print(emotion_labels)
for i in range(2):
    print(test_labels[i],"\n",predictions[i], "\n\n")
#print(test_labels, "\n\n", predictions)

['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']
[0 0 0 0 1 0 1 0 0 0 0] 
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0] 


[1 0 1 0 0 0 0 0 1 0 0] 
 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] 




In [27]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

In [28]:
base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
        return_dict=True,
        low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
# merge adopter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)
model = model.merge_and_unload()

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [30]:
#push the model to hub

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/AbidaRashid/mistral_7b_mlec/commit/0b6171075ba3d4d4c113a89f0850c12d20950f0d', commit_message='Upload tokenizer', commit_description='', oid='0b6171075ba3d4d4c113a89f0850c12d20950f0d', pr_url=None, pr_revision=None, pr_num=None)